In [1]:
from collections import namedtuple
import re

import pdfplumber
import pandas as pd

In [2]:
Line = namedtuple('Line', 'No Article Desc Quant UOM MRP BaseCost IGST_perc IGST_INR Total_Base HSN_SAC_Code Site')

In [3]:
line_re = re.compile(r'\d \d{2,}')

In [4]:
def numbify(num):
    return float(num.replace('$', '').replace(',', ''))

In [8]:
with pdfplumber.open("samp.pdf") as pdf:
    page = pdf.pages[0]
    text = page.extract_text(x_tolerance=2, y_tolerance=0)

In [10]:
data = []

with pdfplumber.open("samp.pdf") as pdf:
    page = pdf.pages[0]
    text = page.extract_text(x_tolerance=2, y_tolerance=0)
    
    for line in text.split('\n'):
        if line_re.search(line):
            in_lines = True
            no, article, *desc, quant, uom, mrp, basecost, igstp, igst_inr, total_base = line.split()
            desc = ' '.join(desc)
        elif line.startswith('Grand'):
            break
        elif re.match(r'\d{4}', line):
            hsn_code = line
        elif re.match(r'T\S{3}', line):
            site = line
            line_info = Line(no, article, desc, quant, uom, mrp, basecost, igstp, igst_inr, total_base, hsn_code, site)
            data.append(line_info)
df = pd.DataFrame(data)
df

,No,Article,Desc,Quant,UOM,MRP,BaseCost,IGST_perc,IGST_INR,Total_Base,HSN_SAC_Code,Site
0,1,310022091,PRINT-COUPON-500 6X4IN 4COL OFFSET130GSM,"3,500",EA,0.44,0.44,12.00,184.80,"1,540.00",4911,TDN6
1,2,310022091,PRINT-COUPON-500 6X4IN 4COL OFFSET130GSM,"1,000",EA,0.44,0.44,12.00,52.80,440.00,4911,TEG3
2,3,310022091,PRINT-COUPON-500 6X4IN 4COL OFFSET130GSM,"2,000",EA,0.44,0.44,12.00,105.60,880.00,4911,TX06


In [11]:
df['Total_Base'] = df['Total_Base'].map(numbify)

In [12]:
sum(df['Total_Base'])

2860.0

In [13]:
df['IGST_INR'] = df['IGST_INR'].map(numbify)
sum(df['IGST_INR'])

343.20000000000005

In [14]:
df.to_csv('file.csv', index=False)